In [1]:
!pip install anndata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 2.1 MB/s eta 0:00:00


In [2]:
import os

from google.colab import drive

import anndata


drive.mount('/content/drive') # use if you plan to use colab.

Mounted at /content/drive


In [3]:
PATH_TO_FOLDER = "/content/drive/MyDrive/DLS/train"
TRAIN_DATA_PATH = '/content/drive/MyDrive/DLS/train'
ORIGINAL_IMAGE_DATA_SUBDIR = 'images_masks'
ORIGINAL_MASKS_SUBDIR = 'masks'
ORIGINAL_IMAGES_SUBDIR = 'img'

if PATH_TO_FOLDER is None:
    raise ValueError('Please set PATH_TO_FOLDER to a path with unzipped training data.')

ANNDATA_PATH = 'cell_data.h5ad'
TRAIN_ANNDATA_PATH = os.path.join(TRAIN_DATA_PATH, ANNDATA_PATH)
TRAIN_IMAGE_DATA_DIR = os.path.join(TRAIN_DATA_PATH, ORIGINAL_IMAGE_DATA_SUBDIR)
TRAIN_IMAGE_DATA_IMAGES = os.path.join(TRAIN_IMAGE_DATA_DIR, ORIGINAL_IMAGES_SUBDIR)
TRAIN_IMAGE_DATA_MASKS = os.path.join(TRAIN_IMAGE_DATA_DIR, ORIGINAL_MASKS_SUBDIR)

In [4]:
adata = anndata.read_h5ad(TRAIN_ANNDATA_PATH)

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pandas as pd
import seaborn as sns

In [6]:
adata.obs.columns

Index(['image', 'sample_id', 'ObjectNumber', 'Pos_X', 'Pos_Y', 'area',
       'major_axis_length', 'minor_axis_length', 'eccentricity', 'width_px',
       'height_px', 'acquisition_id', 'SlideId', 'Study', 'Box.Description',
       'Position', 'SampleId', 'Indication', 'BatchId', 'SubBatchId', 'ROI',
       'ROIonSlide', 'includeImage', 'flag_no_cells', 'flag_no_ROI',
       'flag_total_area', 'flag_percent_covered', 'small_cell', 'celltypes',
       'flag_tumor', 'PD1_pos', 'Ki67_pos', 'cleavedPARP_pos', 'GrzB_pos',
       'tumor_patches', 'distToCells', 'CD20_patches', 'Batch', 'cell_labels'],
      dtype='object')

In [7]:
marker_names = list(adata.var['marker'])
cell_labels_exprs = adata.obs['cell_labels']
df = pd.DataFrame(adata.layers['exprs'], columns = marker_names)
df_all = pd.concat([df,adata.obs[['Pos_X','Pos_Y','ROI','image']].reset_index(drop=True)], axis=1)
df_all


,MPO,HistoneH3,SMA,CD16,CD38,HLADR,CD27,CD15,CD45RA,CD163,...,Ecad,CD303,CD206,cleavedPARP,DNA1,DNA2,Pos_X,Pos_Y,ROI,image
0,0.000000,1.819988,0.502650,1.265410,0.515007,3.331261,0.403800,0.455421,0.528734,2.948392,...,0.777378,0.000000,3.581082,0.424273,4.761130,5.301351,300.846154,0.692308,2,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...
1,0.135920,2.393087,1.136961,0.505529,0.605836,0.710857,0.419814,0.100110,0.743002,0.329851,...,1.884778,0.068872,0.744634,0.118620,5.029241,5.530764,26.982143,0.928571,2,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...
2,0.165905,2.244917,1.396334,1.744382,0.458541,4.006500,0.003607,0.562882,1.236516,1.747839,...,1.743507,0.403610,2.244398,0.015141,4.418390,4.815923,309.083333,0.750000,2,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...
3,0.165905,1.144778,0.563216,1.166010,0.513735,2.034705,0.650242,0.250150,1.205007,0.990385,...,2.268924,0.196199,0.946492,0.233095,3.736089,4.368012,431.916667,0.750000,2,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...
4,0.137497,2.210970,0.892371,0.497646,0.412840,1.499184,0.098131,0.208566,0.922874,1.255892,...,2.387292,0.061568,1.632904,0.372278,4.607359,5.152041,116.931034,1.206897,2,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236786,0.223930,5.665851,2.734667,2.338794,0.853400,1.660905,1.660581,1.202203,1.322045,0.789409,...,1.382112,0.813415,0.531336,0.034725,4.334035,4.838697,596.548387,596.709677,2,IMMUcan_Batch20220908_S-220729-00002_002.tiff
236787,0.149443,3.908886,2.751400,0.702922,0.539905,0.435384,0.557881,0.146520,0.402526,0.317224,...,1.200277,0.230447,0.203544,0.000000,4.633061,5.182098,180.300000,597.400000,2,IMMUcan_Batch20220908_S-220729-00002_002.tiff
236788,0.110884,3.520289,0.310834,2.590231,2.336469,3.652604,1.443007,0.248852,0.455855,1.025672,...,1.427727,0.994726,0.224414,0.102515,4.303070,4.826098,48.370370,598.111111,2,IMMUcan_Batch20220908_S-220729-00002_002.tiff
236789,0.150941,4.214958,2.651646,0.807826,0.403674,1.226701,0.400776,0.265660,0.513530,0.168271,...,0.792030,0.047485,0.090598,0.083975,4.822456,5.361662,207.969697,598.060606,2,IMMUcan_Batch20220908_S-220729-00002_002.tiff


In [8]:
df_all['ROI'].unique()

array([2, 3, 1, 4])

In [9]:
df_all['image'].nunique()

125

In [10]:
for img in df_all['image'].unique():
    n_rois = df_all[df_all['image']==img]['ROI'].nunique()
    if n_rois > 1:
        print(f"No. of ROIs for image {img} is > 1 ({n_rois})")

For every image there is only one ROI value.

In [17]:
celesta_input_dir = "/content/drive/MyDrive/DLS/celesta_input"
if not os.path.exists(celesta_input_dir):
    os.mkdir(celesta_input_dir)

In [14]:
extract_celesta_input(df_all, img, roi, celesta_cols, celesta_input_dir)

In [15]:
def extract_celesta_input(df, image, roi, col_names, dir):
    df_subset = df[(df['image']==image) & (df['ROI']==roi)][col_names]
    img_name = image.split(".")[0]
    df_subset.to_csv(os.path.join(dir,f"{img_name}_roi_{roi}.csv"), index=False)

extract_celesta_input(df_all, img, roi, celesta_cols, ".")

In [19]:
def celesta_input(adata, dir):
    marker_names = list(adata.var['marker'])
    df = pd.DataFrame(adata.layers['exprs'], columns = marker_names)
    df = pd.concat([df,adata.obs[['Pos_X','Pos_Y','ROI','image']].reset_index(drop=True)], axis=1)
    df.rename(columns={"Pos_Y":"Y", "Pos_X":"X"}, inplace=True)
    celesta_cols = ['X','Y'] + marker_names
    print(df['image'].nunique())

    for img in df['image'].unique():
        for roi in df[df['image']==img]['ROI'].unique():
            # print(img, roi)
            extract_celesta_input(df, img, roi, celesta_cols, dir)

celesta_input(adata, celesta_input_dir)




125
